# Airyjevi funkciji
#### Matematično-fizikalni praktikum, oktober 2023

*Luka Skeledžija, 28201079*

<style>
.MJXc-display{
    display: inline-block !important;
    width: 100%;
}
@media print {
    .pagebreak { page-break-before: always; } /* page-break-after works, as well */
}

img{
    width: 100%;
    max-width: 600px !important;
    margin: auto !important;
}

body {
    overflow: hidden;
    max-width: 600px;
    margin: auto;
}

::-webkit-scrollbar {
  width: 0px;
}

table{
    width: 100%;
}


</style>




# Uvod

Airyjevi funkciji $\text{Ai}$ in $\text{Bi}$ se v fiziki pojavljata predvsem v optiki in kvantni mehaniki. Definirani sta kot neodvisni rešitvi enačbe

$$   y''(x) -xy(x) = 0, $$ 

ki je znana kot Airyjeva oz. Stokesova enačba. To je najenostavnejša linearna diferencialna enačba drugega reda z obratno točko (tj. točko, kjer se značaj rešitev spremeni iz oscilatornega v eksponentni). Neodvisni rešitvi enačbe sta predstavljivi v integralski obliki kot

$$   \text{Ai}(x) = \frac{1}{\pi} \int_0^\infty \cos (t^3/3 + x t) \,\mathrm{d} t \>,\quad $$ 

$$   \text{Bi}(x) = \frac{1}{\pi} \int_0^\infty \left[ \mathrm{e}^{-t^3/3 + x t}
  + \sin (t^3/3 + x t) \right] \,\mathrm{d} t \>. $$

# Naloga

Z uporabo kombinacije Maclaurinove vrste in asimptotskega razvoja poišči čim učinkovitejši postopek za izračun
vrednosti Airyjevih funkcij $\text{Ai}$ in $\text{Bi}$ na vsej realni osi
z **absolutno** napako, manjšo od $10^{-10}$. Enako naredi tudi z **relativno** napako in ugotovi,
ali je tudi pri le-tej dosegljiva natančnost, manjša od $10^{-10}$.
Pri oceni napak si pomagaj s programi, ki znajo računati s poljubno
natančnostjo.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

x = np.linspace(-15, 15, 1000)
print(x)

[-15.         -14.96996997 -14.93993994 -14.90990991 -14.87987988
 -14.84984985 -14.81981982 -14.78978979 -14.75975976 -14.72972973
 -14.6996997  -14.66966967 -14.63963964 -14.60960961 -14.57957958
 -14.54954955 -14.51951952 -14.48948949 -14.45945946 -14.42942943
 -14.3993994  -14.36936937 -14.33933934 -14.30930931 -14.27927928
 -14.24924925 -14.21921922 -14.18918919 -14.15915916 -14.12912913
 -14.0990991  -14.06906907 -14.03903904 -14.00900901 -13.97897898
 -13.94894895 -13.91891892 -13.88888889 -13.85885886 -13.82882883
 -13.7987988  -13.76876877 -13.73873874 -13.70870871 -13.67867868
 -13.64864865 -13.61861862 -13.58858859 -13.55855856 -13.52852853
 -13.4984985  -13.46846847 -13.43843844 -13.40840841 -13.37837838
 -13.34834835 -13.31831832 -13.28828829 -13.25825826 -13.22822823
 -13.1981982  -13.16816817 -13.13813814 -13.10810811 -13.07807808
 -13.04804805 -13.01801802 -12.98798799 -12.95795796 -12.92792793
 -12.8978979  -12.86786787 -12.83783784 -12.80780781 -12.77777778
 -12.74774